In [1]:
import pandas as pd

# Load the CSV files
#data1 = pd.read_csv('/home/ws/drones_control/px4/simulation/pose_estimation/imu_rpy_odometry_data_valongo.csv')
#data2 = pd.read_csv('/home/ws/drones_control/px4/simulation/pose_estimation/imu_rpy_odometry_data_prague.csv')

# Concatenate the datasets
#data = pd.concat([data1, data2], ignore_index=True)

data = pd.read_csv('imu_odometry_data.csv')

# Calculate incremental pose (delta values)
data['delta_x'] = data['x'].diff()
data['delta_y'] = data['y'].diff()
data['delta_z'] = data['z'].diff()
data['delta_roll'] = data['roll'].diff()
data['delta_pitch'] = data['pitch'].diff()
data['delta_yaw'] = data['yaw'].diff()

# Drop the first row (NaN values from diff)
data = data.dropna()

# Drop rows where any of the IMU accelerometer readings are smaller than 0.5 in absolute value
#data = data[
#    (data['imu_accel_x'].abs() >= 3.0) & 
#    (data['imu_accel_y'].abs() >= 3.0) & 
#    (data['imu_accel_z'].abs() >= 3.0)
#]

# Features (IMU data)
X = data[['imu_accel_x', 'imu_accel_y', 'imu_accel_z', 'imu_gyro_x', 'imu_gyro_y', 'imu_gyro_z', 'roll', 'pitch', 'yaw']]

# Targets (incremental pose)
y = data[['delta_x', 'delta_y', 'delta_z', 'delta_roll', 'delta_pitch', 'delta_yaw']]

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
model.fit(X_train, y_train)

# Predict on the test set
y_pred = model.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse}')


Mean Squared Error: 0.026717515193329374


In [2]:
import joblib

# Save the Random Forest model
joblib.dump(model, 'pose_estimation_model.pkl')

# To load the model later:
# model = joblib.load('pose_estimation_model.pkl')


['pose_estimation_model.pkl']